In [ ]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
import json
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

## Load app into project

In [ ]:
# need to have converted app to json first using rabix!
import json
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
f = open('/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/kfdrc_annoFuse_wf.json', 'r')
app_raw = f.read()
app = json.loads(app_raw)
app_id = "kfdrc-annofuse-wf"
# Create the Workflows
a_id = (project + "/" + app_id)
my_app_first = api.apps.install_app(id = a_id, raw = app)

## Create example bash calls from workflow
Works best with non-restarted WF

In [ ]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
task_id = "00210a5f-77ec-4d07-9b1d-c08e5497e24c"
out_file = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08-26_gatk4_rpt.tsv", "w")
# task_id = "3c20cc8e-18d7-43f2-bc2c-4a76d38a88f8"
task = api.tasks.get(task_id)
jobs = {}
temp = {}
for job in task.get_execution_details().jobs:
    if job.status == "COMPLETED":
        check = job.name.split('_')
        cmd = job.command_line
        if job.command_line == None:
            # pdb.set_trace()
            cmd = "embedded script or task retry"
            sys.stderr.write("WARN: Job " + job.name + " had null cmd\n")
        if check[-1] == "s":
            key = "_".join(check[:-2])
            if key not in temp:
                jobs[job.start_time] = {}
                jobs[job.start_time][key] = cmd
                temp[key] = 1
            else:
                temp[key] += 1
        else:
            jobs[job.start_time] = {}
            jobs[job.start_time][job.name] = cmd
out_file.write("Step\tType\tNum scatter\tCommand\n")
for rtime in sorted(jobs.keys()):
    for key in jobs[rtime]:
        rtype = "run step"
        sct = "NA"
        if key in temp and temp[key] > 1:
            rtype = "scatter"
            sct = str(temp[key])
        cmds = jobs[rtime][key].split('\n')
        for cmd in cmds:
            out_file.write(key + "\t" + rtype + "\t" + sct + "\t" + cmd + "\n")
out_file.close()


## Convert tsv to markdown table

In [ ]:
import sys

# max desired col width
max_w = 200
tsv_in = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08-26_gatk4_rpt.tsv")
out_md = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08-26_gatk4_rpt.md", "w")
data = []
max_len = []


for line in tsv_in:
    info = line.rstrip('\n').split('\t')
    data.append(info)
    if len(max_len) == 0:
        for item in info:
            max_len.append(len(item))
    else:
        for i in range(len(max_len)):
            if len(info[i]) > max_w:
                max_len[i] = max_w
            elif len(info[i]) > max_len[i]:
                max_len[i] = len(info[i])
# print header first
d_ct = []
for i in range(len(data[0])):
    d_ct.append(len(data[0][i]))
    out_md.write(" | " + data[0][i] + "".join([" "] * max_len[i]))
    d_ct[i] += max_len[i]
out_md.write(" |\n")
for i in range(len(data[0])):
    out_md.write(" | " + "".join(["-"] * d_ct[i]))
out_md.write(" |\n")
# pdb.set_trace()
for i in range(1, len(data), 1):
    for j in range(len(data[i])):
        d_ct = len(data[i][j]) + 2
        out_md.write(" | " + data[i][j] + "".join([" "] * max_len[j]))
        d_ct += max_len[j]
    out_md.write(" |\n")
out_md.close()

## Get run times

### Get run times by step

In [ ]:
def get_job_run_time(task, phrase):
    data = []
    if re.search(phrase, task.name):
        try:
            for job in task.get_execution_details().jobs:
                if job.status != "COMPLETED":
                    sys.stderr.write("Skipping job likely killed due to spot instance kill for " + job.name + " from task " + task.id + "\n")
                else:
                    data.append([job.name, str((job.end_time-job.start_time).seconds/3600)])
            # pdb.set_trace()
            hold=1
            return task.id, task.name, str(task.price.amount), str((task.end_time - task.start_time).seconds/3600), data
        except Exception as e:
            return [e, task.id]
    else:
        return []


In [ ]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
phrase = "GATK RNAseq SNV RPT"

tasks = api.tasks.query(project=project, status="COMPLETED").all()
actual_out = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/cost_est/gatk4_rpt_cov-irt-actual_cost.txt", "w")
actual_out.write("Task name\tTask ID\tCost\tRun Time in hours\n")
step_run = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/cost_est/gatk4_rpt_cov-irt_step_run_times.txt", "w")
step_run.write("Run step\tRun time in hours\n")
# for task in tasks:
#     result = get_job_run_time(task, phrase)
#     if len(result) > 0:
#         pdb.set_trace()
#         actual_out.write("\t".join(result[0:4]) + "\n")
#         for step in result[4]:
#             step_run.write("\t".join(step) + "\n")
x = 1
m = 100
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(get_job_run_time, task, phrase): task for task in tasks}
    for result in concurrent.futures.as_completed(results):
        if len(result.result()) > 2:
            if x % m == 0:
                sys.stderr.write("Processed " + str(x) + " valid tasks\n")
            actual_out.write("\t".join(result.result()[0:4]) + "\n")
            for step in result.result()[4]:
                step_run.write("\t".join(step) + "\n")
            x += 1
        elif len(result.result()) == 2:
            sys.stderr.write(str(result.result()[0]) + "\tFailed processing task ID " + result.result()[1] + "\n")
            exit(1)
actual_out.close()
step_run.close()


## Tag source files

In [ ]:
def tag_file(info, header):
    try:
        meta = info.rstrip('\n').split('\t')
        f_obj = api.files.get(meta[0])
        metadata = {}
        for i in range(3, len(header), 1):
            metadata[header[i]] = meta[i]
        f_obj.metadata = metadata
        f_obj.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\n")
        sys.stderr.write("Could not process " + info)
        exit(1)

In [ ]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/rsem_amnifest_to_tag.tsv")
head = next(manifest)
header = head.rstrip("\n").split("\t")
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(tag_file, line, header): line for line in manifest}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' files\n')
            sys.stderr.flush()


## Set up GATK4 RNAseq WF Tasks

In [ ]:
def get_gatk_refs(api, project):
    try:
        ref_dict = {}
        ref_dict['reference_fasta'] = api.files.get('5f185f0de4b09d9af8ae456e')
        ref_dict['reference_dict'] = api.files.get('5f185f09e4b09d9af8ae4569')
        known_sites = []
        known_sites.append(api.files.get('5f161613e4b0efd84a0fd4b8'))
        known_sites.append(api.files.get('5f1615e3e4b0efd84a0fd4a9'))
        ref_dict['knownsites'] = known_sites
        ref_dict['call_bed_file'] = api.files.get('5f186055e4b09d9af8ae4585')
        ref_dict['dbsnp_vcf'] = api.files.get('5f161572e4b0efd84a0fd49f')
        ref_dict['tool_name'] = 'STAR_GATK4'
    except Exception as e:
        sys.stderr.write(str(e) + "\nFailed to get REFS\n")
        exit(1)
    return ref_dict

In [ ]:
def draft_task(in_file):
    try:
        input_dict = {}
        for key in ref_obj:
            input_dict[key] = ref_obj[key]
        info = in_file.rstrip('\n').split('\t')
        input_dict['STAR_sorted_genomic_bam'] = api.files.get(info[0])
        task_name = "GATK RNAseq SNV RPT: " + info[3]
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nfailed to set up task for " + in_file)
        exit(1)

In [ ]:
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'
app_name = project + "/d3b-gatk-rnaseq-snv-wf"
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08-10_bam_list.tsv")
head = next(manifest)
ref_obj = get_gatk_refs(api, project)
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(draft_task, line ): line for line in manifest}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' tasks\n')
            sys.stderr.flush()


## Run VEP

In [ ]:
def get_vep_refs(api, project):
    try:
        ref_dict = {}
        ref_dict['reference'] = api.files.get('5f185f0de4b09d9af8ae456e')
        # un comment for using cache
        # ref_dict['cache'] = api.files.get('5eed0f54e4b0efd899f4afda')
        # ref_dict['merged_cache'] = True
        ref_dict['bgzipped_gtf'] = api.files.get('5f3550c2e4b0efd8002a853b')
        ref_dict['tool_name'] = 'STAR_GATK4'
    except Exception as e:
        sys.stderr.write(str(e) + "\nFailed to get REFS\n")
        exit(1)
    return ref_dict

In [ ]:
def draft_vep_task(in_file):
    try:
        input_dict = {}
        for key in ref_obj:
            input_dict[key] = ref_obj[key]
        info = in_file.rstrip('\n').split(',')
        input_dict['input_vcf'] = api.files.get(info[0])
        task_name = "VEP R100 GTF ANNOTATE RPT: " + info[sidx]
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nfailed to set up task for " + in_file)
        exit(1)

In [ ]:
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'
app_name = project + "/vep-1oo-annotate"
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08_manifests/vcf-manifest.csv")
head = next(manifest)
header = head.rstrip('\n').split(',')
sidx = header.index('sample_id')
ref_obj = get_vep_refs(api, project)
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(draft_vep_task, line ): line for line in manifest}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' tasks\n')
            sys.stderr.flush()
        x += 1


# Run STAR Fusion

In [ ]:
def get_sf_refs(api, project):
    try:
        ref_dict = {}
        ref_dict['genome_tar'] = api.files.get('5f19e9cee4b0a6d31720b606')
        ref_dict['genome_untar_path'] = 'ctat_genome_lib_build_dir'
    except Exception as e:
        sys.stderr.write(str(e) + "\nFailed to get REFS\n")
        exit(1)
    return ref_dict

In [ ]:
def draft_sf_task(in_file):
    try:
        input_dict = {}
        for key in ref_obj:
            input_dict[key] = ref_obj[key]
        info = in_file.rstrip('\n').split(',')
        input_dict['Chimeric_junction'] = api.files.get(info[0])
        task_name = "STAR FUSION: " + info[sidx]
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
        task.inputs['SampleID'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nfailed to set up task for " + in_file)
        exit(1)

In [ ]:
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'
app_name = project + "/star-fusion-covirt"
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08_manifests/chim_junction-manifest.csv")
head = next(manifest)
header = head.rstrip('\n').split(',')
sidx = header.index('sample_id')
ref_obj = get_sf_refs(api, project)
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(draft_sf_task, line ): line for line in manifest}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' tasks\n')
            sys.stderr.flush()
        x += 1


# Run arriba

In [ ]:
def get_arriba_refs(api, project):
    try:
        ref_dict = {}
        ref_dict['reference_fasta'] = api.files.get('5f185f0de4b09d9af8ae456e')
        ref_dict['gtf_anno'] = api.files.get('5f186055e4b09d9af8ae4585')
    except Exception as e:
        sys.stderr.write(str(e) + "\nFailed to get REFS\n")
        exit(1)
    return ref_dict

In [ ]:
def draft_arriba_task(samp_id):
    try:
        input_dict = {}
        for key in ref_obj:
            input_dict[key] = ref_obj[key]
        for key in inputs[samp_id]:
            input_dict[key] = api.files.get(inputs[samp_id][key])
        task_name = "ARRIBA FUSION: " + samp_id
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
        task.inputs['outFileNamePrefix'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nfailed to set up task for " + in_file)
        exit(1)

In [ ]:
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'
app_name = project + "/arriba-fusion"
inputs = {}
# process two manifests, chimeric_sam, genome bam + bai, 
chim_sam = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08_manifests/chimeric_sam-manifest.csv")
head = next(chim_sam)
header = head.rstrip('\n').split(',')
sidx = header.index('sample_id')
for line in chim_sam:
    info = line.rstrip('\n').split(',')
    inputs[info[sidx]] = {}
    inputs[info[sidx]]['chimeric_sam_out'] = info[0]
chim_sam.close()
ba_manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08_manifests/bam_bai.csv")
head = next(ba_manifest)
header = head.rstrip('\n').split(',')
sidx = header.index('sample_id')
for line in ba_manifest:
    info = line.rstrip('\n').split(',')
    suffix = info[1][-3:]
    inputs[info[sidx]][('genome_aligned_' + suffix)] = info[0]
ba_manifest.close()

ref_obj = get_arriba_refs(api, project)
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(draft_arriba_task, samp_id ): samp_id for samp_id in inputs}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' tasks\n')
            sys.stderr.flush()
        x += 1
# for samp_id in inputs:
#     draft_arriba_task(samp_id)


# Run annoFuse

In [ ]:
def get_af_refs(api, project):
    try:
        ref_dict = {}
        ref_dict['FusionGenome'] = api.files.get('5f19e9cee4b0a6d31720b606')
        ref_dict['genome_untar_path'] = 'ctat_genome_lib_build_dir'
    except Exception as e:
        sys.stderr.write(str(e) + "\nFailed to get REFS\n")
        exit(1)
    return ref_dict

In [ ]:
def draft_annofuse_task(samp_id):
    try:
        input_dict = {}
        for key in ref_obj:
            input_dict[key] = ref_obj[key]
        for key in inputs[samp_id]:
            input_dict[key] = api.files.get(inputs[samp_id][key])
        input_dict['sample_name'] = samp_id
        task_name = "annoFuse: " + samp_id
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nfailed to set up task for " + in_file)
        exit(1)

In [ ]:
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'
app_name = project + "/kfdrc-annofuse-wf"
inputs = {}
# process two manifests, fusion files, rsem, 
rsem_files = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08-20_rsem_annfuse_manifest.tsv")
head = next(rsem_files)
header = head.rstrip('\n').split('\t')
sidx = header.index('sample_id')
for line in rsem_files:
    info = line.rstrip('\n').split('\t')
    inputs[info[sidx]] = {}
    inputs[info[sidx]]['rsem_expr_file'] = info[0]
rsem_files.close()
fusions = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-08-20_fusion_results-manifest.csv")
head = next(fusions)
header = head.rstrip('\n').split(',')
sidx = header.index('sample_id')
for line in fusions:
    info = line.rstrip('\n').split(',')
    key = 'arriba_output_file'
    if re.search("STAR", info[1]):
        key = 'star_fusion_output_file'
    inputs[info[sidx]][key] = info[0]
fusions.close()

ref_obj = get_af_refs(api, project)
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(draft_annofuse_task, samp_id ): samp_id for samp_id in inputs}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' tasks\n')
            sys.stderr.flush()
        x += 1

## Copy metadata to outputs

In [ ]:
def add_metadata_to_outputs(task, phrase, in_key):
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        metadata = {}
        for key in task.inputs[in_key].metadata:
            metadata[key] = task.inputs[in_key].metadata[key]
        for out_key in task.outputs:
            # pdb.set_trace()
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    file_obj.metadata = metadata
                    file_obj.save()
                    try:
                        if task.outputs[out_key].secondary_files is not None:
                            file_obj = api.files.get(task.outputs[out_key].secondary_files[0].id)
                            file_obj.metadata = metadata
                            file_obj.save()
                    except Exception as e:
                        sys.stderr.write(str(e) + "\nError processing secondary file for " + out_key + " in " + task.id + " skipping\n")

                else:
                    for output in task.outputs[out_key]:
                        if type(output) is not list:
                            file_obj = api.files.get(output.id)
                            file_obj.metadata = metadata
                            file_obj.save()
                        else:
                            for item in output:
                                if item is not None:
                                    file_obj = api.files.get(item.id)
                                    file_obj.metadata = metadata
                                    file_obj.save()

            except Exception as e:
                print(e)
                print("Skipping " + out_key + " for " + task.name + " due to error")

In [ ]:
prefix = 'VEP R100 GTF ANNOTATE RPT'
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
key = 'input_vcf'
print("You sure you want to transfer input metadata to outputs with task prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":
    tasks = api.tasks.query(project=project, status="COMPLETED").all()
    #for task in tasks:
    #    add_metadata_to_outputs(task, prefix, key)
    x = 1
    m = 250
    with concurrent.futures.ThreadPoolExecutor(16) as executor:
        results = {executor.submit(add_metadata_to_outputs, task, prefix, key ): task for task in tasks}

else:
    sys.stderr.write("User did not type YASS, skipping\n")

### Add tags to task outputs

In [ ]:
def add_tags_to_outputs(task, phrase, tags):
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        for out_key in task.outputs:
            # pdb.set_trace()
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    file_obj.tags = tags
                    file_obj.save()
                    try:
                        if task.outputs[out_key].secondary_files is not None:
                            file_obj = api.files.get(task.outputs[out_key].secondary_files[0].id)
                            file_obj.tags = tags
                            file_obj.save()
                    except Exception as e:
                        sys.stderr.write(str(e) + "\nError processing secondary file for " + out_key + " in " + task.id + " skipping\n")

                else:
                    for output in task.outputs[out_key]:
                        if type(output) is not list:
                            file_obj = api.files.get(output.id)
                            file_obj.tags = tags
                            file_obj.save()
                        else:
                            for item in output:
                                if item is not None:
                                    file_obj = api.files.get(item.id)
                                    file_obj.tags = tags
                                    file_obj.save()

            except Exception as e:
                print(e)
                print("Skipping " + out_key + " for " + task.name + " due to error")

In [ ]:
prefix = 'VEP R100 GTF ANNOTATE'
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tags = ['GATK4', 'VEP', 'R100', 'GTF-ANNOTATED']
print("You sure tag outputs with task prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":
    tasks = api.tasks.query(project=project, status="COMPLETED").all()
    #for task in tasks:
    #    add_metadata_to_outputs(task, prefix, key)
    x = 1
    m = 250
    with concurrent.futures.ThreadPoolExecutor(16) as executor:
        results = {executor.submit(add_tags_to_outputs, task, prefix, tags ): task for task in tasks}

else:
    sys.stderr.write("User did not type YASS, skipping\n")

## Get task outputs

In [ ]:
def write_to_manifest(out_fh, file_obj, out_key, task_name):
    out_fh.write(",".join([file_obj.id, file_obj.name, out_key, task_name]) + "\n")

In [ ]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="COMPLETED").all()
out = open('/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/manifests/test_vep_out.txt', 'w')
out.write("id,name,output_category,task_name\n")
phrase = "VEP R100 ANNOTATE: COVHA-20200311-P1-A01-CP"

for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Processing task: ' + task.name + "\n")
        for out_key in task.outputs:
            if type(task.outputs[out_key]) is not list:
                file_obj = task.outputs[out_key]
                write_to_manifest(out, file_obj, out_key, task.name)
                if task.outputs[out_key].secondary_files is not None:
                    write_to_manifest(out, task.outputs[out_key].secondary_files[0], out_key, task.name)
            else:
                for i in range(len(task.outputs[out_key])):
                    if type(task.outputs[out_key][i]) is not list:
                        write_to_manifest(out, task.outputs[out_key][i], out_key, task.name)
                        if task.outputs[out_key][i].secondary_files is not None:
                            write_to_manifest(out, task.outputs[out_key][i].secondary_files[0], out_key, task.name)
                    else:
                        for j in range(len(task.outputs[out_key][i])):
                            if task.outputs[out_key][i][j] is not None:
                                write_to_manifest(out, task.outputs[out_key][i][j], out_key, task.name)
                                if task.outputs[out_key][i][j].secondary_files is not None:
                                    write_to_manifest(out, task.outputs[out_key][i][j].secondary_files[0], out_key, task.name)
out.close()


## Abort unresponsive and restart tasks

In [ ]:
import datetime
import pytz

project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="RUNNING")
current = datetime.datetime.now()
tz = pytz.timezone('America/New_York')
prefix = "GATK RNAseq SNV"
task_abort = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/TASK_RUN/aborted_and_restarted2.log", 'w')
task_abort.write("Task ID\tTask name\tNew Task ID")
for task in tasks:
    if re.search(prefix, task.name):
        for job in task.get_execution_details().jobs:
            if job.name == "preprocess_rnaseq_bam_sambamba_md_sorted":
                if job.status == "RUNNING":
                    diff = (current-pytz.utc.localize(job.start_time, is_dst=None).astimezone(tz).replace(tzinfo=None)).seconds/3600
                    if diff > 2:
                        task_abort.write(task.id + "\t" + task.name)
                        in_dict = {}
                        sys.stderr.write("Aborting " + task.id + "\t" + task.name + "\n" )
                        task.abort()
                        new_task = task.clone(run=False)
                        new_task.inputs['output_basename'] = new_task.id
                        new_task.save()
                        task_abort.write("\t" + new_task.id + "\n")
                        task_abort.flush()
                    else:
                        break
                else:
                    break
task_abort.close()

## Restart failed tasks

In [ ]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="FAILED")
prefix = "GATK RNAseq SNV"
task_restart = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/TASK_RUN/failed_and_restarted2.log", 'w')
task_restart.write("Task ID\tTask name\tNew Task ID\n")
run_list = ["GATK RNAseq SNV: COVHA-20200315-P9-F02-N",
             "GATK RNAseq SNV: COVHA-20200316-P12-F01-P","GATK RNAseq SNV: COVHA-20200403-P1-C06-P"]
for task in tasks:
    if re.search(prefix, task.name) and task.name in run_list:
        task_restart.write(task.id + "\t" + task.name)
        new_task = task.clone(run=False)
        new_task.inputs['output_basename'] = new_task.id
        new_task.save()
        task_restart.write("\t" + new_task.id + "\n")
        task_restart.flush()
task_restart.close()


### Get Failed list

In [ ]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="FAILED")
prefix = "GATK RNAseq SNV"
task_failed = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/TASK_RUN/failed.log", 'w')
task_failed.write("Task ID\tTask name\n")
for task in tasks:
    if re.search(prefix, task.name):
        task_failed.write(task.id + "\t" + task.name + "\n")
task_failed.close()

### Remove outputs from failed and aborted tasks

In [ ]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="ABORTED")
prefix = "GATK RNAseq SNV"
print("You sure remove outputs from failed tasks with prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":

    for task in tasks:
        if re.search(prefix, task.name):
            for key in task.outputs:
                if task.outputs[key] is not None:
                    sys.stderr.write("Found files to remove from failed task: " + task.id + " " + task.name + "\n")
                    try:
                        if task.outputs[key].secondary_files is not None:
                            sys.stderr.write("Removing secondary files\n")
                            for i in range(0, len(task.outputs[key].secondary_files), 1):
                                task.outputs[key].secondary_files[i].delete()
                    except Exception as e:
                        sys.stderr.write(str(e) + "\nFile with key " + key + " probably does not have secondaryFiles, skipping\n")
                    try:
                        task.outputs[key].delete()
                    except Exception as e:
                        sys.stderr.write(str(e) + "\nFile with key " + key + " was probably deleted before, skipping\n")            
                    sys.stderr.write("Finished processing " + task.id + "\n")


### Rename _\d_ files

In [ ]:
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/to_rename.txt")
head = next(manifest)
print("You sure you want to rename the files in that manifest? Type \"YASS\" if so")
check = input()
sep = "\t"
if check == "YASS":
    for line in manifest:
        info = line.split(sep)
        cur = api.files.get(info[0])
        new_name = cur.name[3:]
        sys.stderr.write("Renaming file with ID " + cur.id + " " + cur.name + " to " + new_name + "\n")
        cur.name = new_name
        cur.save()

## Tag outputs by task seq id

In [ ]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="COMPLETED").all()
manifest = open('/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/manifests/2020-06-23_UPDATED_HUMAN_MANIFEST.txt')
head = next(manifest)
header = head.rstrip('\n').split("\t")
phrase = "GATK RNAseq SNV RPT"
meta_dict = {}
for entry in manifest:
    info = entry.rstrip('\n').split('\t')
    meta_dict[info[0]] = {}
    for i in range(0, len(header), 1):
        meta_dict[info[0]][header[i]] = info[i]
manifest.close()
for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Processing task: ' + task.name + "\n")
        parts = task.name.split()
        # Mason lab changed dashes to underscore
        seq_id = parts[-1].replace('-','_' )
        if seq_id in meta_dict:
            for out_key in task.outputs:
                if type(task.outputs[out_key]) is not list:
                    try:
                        file_obj = api.files.get(task.outputs[out_key].id)
                        file_obj.metadata = meta_dict[seq_id]
                        file_obj.save()
                    except Exception as e:
                        sys.stderr.write(str(e) + "\n" + file_obj.name + " probably already tagged, skipping\n" )
                    try:
                        if task.outputs[out_key].secondary_files is not None:
                            file_obj = api.files.get(task.outputs[out_key].secondary_files[0].id)
                            file_obj.metadata = meta_dict[seq_id]
                            file_obj.save()
                    except Exception as e:
                        sys.stderr.write(str(e) + "\nError processing secondary file for " + out_key + " in " + task.id + " skipping\n")

        else:
            sys.stderr.write("Not in manifest: " + task.name + " " + task.id + "\n")


## Delete files by name

In [ ]:
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/delme_files.txt")
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
head = next(manifest)
print("You sure you want to delete the files in that manifest? Type \"YASS\" if so")
check = input()

max_j = 25
ct = 0
found = 0
fnames = []
if check == "YASS":
    for line in manifest:
        fnames.append(line.rstrip('\n'))
        ct +=1
    sys.stderr.write("Searching for " + str(ct) + " files to delete\n")
    total = len(fnames)
    for i in range(0, total, max_j):
        uset = i + max_j
        if uset > total:
            uset = total
        flist = api.files.query(project=project, names=fnames[i:uset])
        for fobj in flist:
            sys.stderr.write("Deleting " + fobj.name + " with ID " + fobj.id)
            fobj.delete()
            found += 1
    sys.stderr.write("Deleted " + str(found) + " files\n")
        